#Imports



In [ ]:
import sys
sys.path.append('../')
from mylibs import *

# Data Loading

In [ ]:
from data.data import Data
handler = Data()
data = handler.load('../artifacts_5s_128hz.pkl')
train_split, val_split, test_split = handler.split(data)

# to-do: modify for CNN-transformer
train = handler.prepare_data(train_split, balance=True, dataset=True)
val = handler.prepare_data(val_split, dataset=True)
test_features, test_labels = handler.prepare_data(test_split)

# Setting up the Hparameter search grid

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

from architectures import 

# Define the hyperparameters you want to tune
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([320, 640]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1, 2,3,4,5]))
HP_NUM_HEADS = hp.HParam('num_heads', hp.Discrete([4,8]))
HP_NUM_CONVLAYERS = hp.HParam('num_convlayers', hp.Discrete([1,2,3,4,5]))

# Define a function to build your model

def create_model(hparams):

    model = TestModel(hparams[HP_NUM_UNITS], hparams[HP_NUM_LAYERS], hparams[HP_NUM_HEADS], hparams[HP_NUM_CONVLAYERS])
    return model


# Define a function to run a single experiment

def run_experiment(hparams, epochs, logdir, savedir, checkpointdir):
    
     # Create the model using the hyperparameters
    model = create_model(hparams)


   
    # Compile the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()])

    # Train the model
    model.fit(train_dataset, 
              epochs=epochs, 
              validation_data = val_dataset,
              callbacks=[tf.keras.callbacks.TensorBoard(logdir),  # log metrics
                        hp.KerasCallback(logdir, hparams),  # log hparams
                        tf.keras.callbacks.ModelCheckpoint(filepath= os.path.join(checkpointdir, "ckpt_{epoch}") ,monitor='val_loss',save_weights_only=True), # save checkpoints when val loss goes down
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)], # early stopping in the case that loss doesnt go down for 3 epochs
              ) 

    

    # Evaluate the model
    accuracy = model.evaluate(test_dataset)

    # Save model parameters
    model.save(savedir)
    return accuracy

# Searching


In [ ]:
# Define the hyperparameter search space

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

session_num = 0
for num_units in HP_NUM_UNITS.domain.values:
    for num_layers in HP_NUM_LAYERS.domain.values:
       for num_heads in HP_NUM_HEADS.domain.values:
         for num_convlayers in HP_NUM_CONVLAYERS.domain.values:
                hparams = {
                    HP_NUM_UNITS: num_units,
                    HP_NUM_LAYERS: num_layers,
                    HP_NUM_HEADS: num_heads,
                    HP_NUM_CONVLAYERS: num_convlayers
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})

                # Run a single experiment
                accuracy = run_experiment(
                    hparams=hparams,
                    epochs = 20,
                    logdir='logs/hparam_tuning/' + run_name, 
                    savedir='logs/models/'+run_name, 
                    checkpointdir='logs/checkpoints'+run_name)
                session_num += 1